# Python & Fortran

You can define Fortran functions and subrouttines in a Jupyter notebook, and call those to process Python data (basic types and numpy arrays).

First, the notebook extension should be installed if that was not already done using pip:

`!pip install fortran-magic`

Once this is done, the extension can be loaded.

In [1]:
%load_ext fortranmagic

/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


We can define a Fortran subroutine.  There are some restrictions on the subroutine's signature since it will behave as a function when called in Python context. The single `intent(out)` argument should not be passed in, and its value will be returned as if it were a function.

In [4]:
%%fortran
subroutine bla(x, r)
    implicit none
    integer, intent(in) :: x
    integer, intent(out) :: r
    r = x + 1
end subroutine

Once the subroutine is defined, it can be used as any Python function.

In [6]:
a = 15
b = 0
b = bla(a)
print(a, b)

15 16
